In [1]:
import os
import numpy as np
import random as rn
import tensorflow as tf

os.environ['PYTHONHASHSEED'] = '0'
np.random.seed(7)
rn.seed(7)

session_conf = tf.ConfigProto(
    intra_op_parallelism_threads=1,
    inter_op_parallelism_threads=1,
    gpu_options=tf.GPUOptions(
        visible_device_list="1", 
        allow_growth=True,
#         per_process_gpu_memory_fraction=0.5
    )
)

from keras import backend as K

tf.set_random_seed(7)
sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
K.set_session(sess)

import pickle
import pandas as pd

import xgboost as xgb
import lightgbm as lgb

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import itertools

from keras.models import load_model
from keras.utils import np_utils

import tensorflow as tf
import pickle
import pandas as pd
import numpy as np

import xgboost as xgb
import lightgbm as lgb
import time

from sklearn.ensemble import RandomForestClassifier as rf
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from keras import models
from keras import layers
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import np_utils
from keras.layers import Dense, Dropout, Activation, Flatten, Input, Dense, GaussianNoise, GaussianDropout
from keras.models import Sequential, Model
from keras.regularizers import l2, l1
from keras.optimizers import Adam, SGD, Nadam
from keras.callbacks import LearningRateScheduler
from keras.metrics import categorical_accuracy
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping
from keras.wrappers.scikit_learn import KerasClassifier
from  keras.regularizers import l1, l2
from keras import backend as K
from keras.utils.generic_utils import get_custom_objects

import matplotlib.pyplot as plt

%matplotlib inline

/home/yoshitaka-i/anaconda3/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# load data
df = pd.concat([
    pd.read_csv('../data/feature_selection_positive.csv', index_col=0),
    pd.read_csv('../data/decomp_pos.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

In [3]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [4]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [5]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [7]:
# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_pos_fs+decomp.h5')

Train on 1340 samples, validate on 336 samples
Epoch 1/100
1340/1340 [==============================] - 2s 1ms/step - loss: 1.9267 - acc: 0.2463 - val_loss: 1.4126 - val_acc: 0.2946
Epoch 2/100
1340/1340 [==============================] - 0s 74us/step - loss: 1.7804 - acc: 0.2582 - val_loss: 1.3865 - val_acc: 0.3065
Epoch 3/100
1340/1340 [==============================] - 0s 76us/step - loss: 1.6781 - acc: 0.2836 - val_loss: 1.4126 - val_acc: 0.3036
Epoch 4/100
1340/1340 [==============================] - 0s 73us/step - loss: 1.7383 - acc: 0.3045 - val_loss: 1.3942 - val_acc: 0.3274
Epoch 5/100
1340/1340 [==============================] - 0s 76us/step - loss: 1.6503 - acc: 0.3037 - val_loss: 1.3520 - val_acc: 0.3542
Epoch 6/100
1340/1340 [==============================] - 0s 70us/step - loss: 1.6012 - acc: 0.3216 - val_loss: 1.3262 - val_acc: 0.4048
Epoch 7/100
1340/1340 [==============================] - 0s 77us/step - loss: 1.5698 - acc: 0.3127 - val_loss: 1.3165 - val_acc: 0.4048
Ep

In [8]:
# load data
df = pd.concat([
    pd.read_csv('../data/feature_selection_negative.csv', index_col=0),
    pd.read_csv('../data/decomp_neg.csv', index_col=0).drop('Subclass', axis=1)
], axis=1)

In [10]:
# divide objective and target
objective = df.Subclass
le = preprocessing.LabelEncoder()
objective = le.fit_transform(objective)

features = df.drop('Subclass', axis=1)

In [11]:
# train test split
random_state=np.random.seed(42)
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    objective,
    test_size=0.2
)

In [12]:
# transform  for keras's target label
y_train_for_keras = np_utils.to_categorical(y_train)
y_test_for_keras = np_utils.to_categorical(y_test)

In [13]:
# make keras model
start = time.time()
inputs = Input(shape=(X_train.shape[1],))

x = Dense(512, activation='relu')(inputs)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)

predictions = Dense(len(df['Subclass'].value_counts()), activation='softmax')(x)
model = Model(inputs=inputs, outputs=predictions)

# compile
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(),
    metrics=['accuracy']
)

# fit
epochs = 100
batch_size = 1000
es = EarlyStopping(monitor='val_loss', patience=20)

history = model.fit(
    X_train,
    y_train_for_keras,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test_for_keras),
    verbose=1,
    callbacks=[
        es,
#         lr_decay
    ]
)

model.save('../model/Keras_ng_fs+decomp.h5')

Train on 373 samples, validate on 94 samples
Epoch 1/100
373/373 [==============================] - 1s 4ms/step - loss: 2.1786 - acc: 0.2279 - val_loss: 1.3114 - val_acc: 0.4149
Epoch 2/100
373/373 [==============================] - 0s 59us/step - loss: 2.0531 - acc: 0.2547 - val_loss: 1.3457 - val_acc: 0.4574
Epoch 3/100
373/373 [==============================] - 0s 54us/step - loss: 1.8356 - acc: 0.2493 - val_loss: 1.4574 - val_acc: 0.4468
Epoch 4/100
373/373 [==============================] - 0s 45us/step - loss: 1.7167 - acc: 0.3217 - val_loss: 1.5464 - val_acc: 0.4681
Epoch 5/100
373/373 [==============================] - 0s 48us/step - loss: 1.8528 - acc: 0.3324 - val_loss: 1.5606 - val_acc: 0.4681
Epoch 6/100
373/373 [==============================] - 0s 54us/step - loss: 1.6720 - acc: 0.3485 - val_loss: 1.5361 - val_acc: 0.4681
Epoch 7/100
373/373 [==============================] - 0s 60us/step - loss: 1.8475 - acc: 0.3566 - val_loss: 1.4950 - val_acc: 0.4681
Epoch 8/100
373/37

Epoch 62/100
373/373 [==============================] - 0s 49us/step - loss: 0.6919 - acc: 0.7158 - val_loss: 0.9997 - val_acc: 0.6064
Epoch 63/100
373/373 [==============================] - 0s 53us/step - loss: 0.6980 - acc: 0.6944 - val_loss: 0.9968 - val_acc: 0.6064
Epoch 64/100
373/373 [==============================] - 0s 51us/step - loss: 0.6963 - acc: 0.7105 - val_loss: 0.9956 - val_acc: 0.6064
Epoch 65/100
373/373 [==============================] - 0s 51us/step - loss: 0.7194 - acc: 0.6863 - val_loss: 0.9929 - val_acc: 0.5957
Epoch 66/100
373/373 [==============================] - 0s 52us/step - loss: 0.6535 - acc: 0.7105 - val_loss: 0.9926 - val_acc: 0.5957


NameError: name 't' is not defined